# XGB kaggle comp

In [15]:
# Load basic libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import statistics as stats
import time

# XGB libraries
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform, randint
from sklearn.preprocessing import StandardScaler


# Import data
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
sample_sub =  pd.read_csv("data/sample_submission.csv")

In [2]:
# Remove NA column from training data
train_df = train_df.drop(columns='Unnamed: 12')

# Assign features
X = train_df.drop(columns='DIC')
y = train_df['DIC']

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=808) 

# Scale the data
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)
X_val_scaled = pd.DataFrame(scaler.transform(X_val), columns=X.columns)

In [8]:
# Determine best number of trees using early stopping
xgb = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.1, 
    early_stopping_rounds=50, 
    eval_metric="rmse",  # Use RMSE for regression
    random_state=808)

# Fit model
xgb.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], verbose=0)

# Print best number of trees
best_ntrees = xgb.best_iteration
print(f"Best number of trees {best_ntrees}")

Best number of trees 305


In [ ]:
# Initialize second XGB to tune learning rate
xgb2 = XGBRegressor(
    n_estimators=best_ntrees,
    early_stopping_rounds=50, 
    eval_metric="rmse", 
    random_state=808)

# Fit model
xgb2.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], verbose=0)

# Create a parameter distribution for learning rate
param_dist = {
    "learning_rate": uniform(0.01, 0.3), 
}

# Set up RandomizedSearchCV
rs = RandomizedSearchCV(
    xgb2, param_dist, n_iter=20, scoring='neg_root_mean_squared_error', 
    cv=3, verbose=0, random_state=808, n_jobs=8
)

# Fit random search
rs.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], verbose=0)

# Print best number of learners
best_lr = rs.best_params_['learning_rate']
print(f"Best learning rate: {best_lr:.4f}")

Best learning rate: 0.1561


In [12]:
# Tune tree specific parameters
xgb3 = XGBRegressor(
    n_estimators = best_ntrees,
    learning_rate = best_lr, 
    early_stopping_rounds=50, 
    eval_metric="rmse", 
    random_state=808)

# Fit model
xgb3.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], verbose=0)

# Second param dist
param_dist2 = {
    "max_depth": randint(3, 10), 
    "min_child_weight": randint(1, 10),
    "gamma": uniform(0.05, 0.05)
}

# Set up RandomizedSearchCV
rs2 = RandomizedSearchCV(
    xgb3, param_dist2, 
    n_iter=20, scoring='neg_root_mean_squared_error', 
    cv=3, verbose=False, random_state=808, n_jobs=10
)

# Run random search
rs2.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], verbose=0)

# Print best tree parameters
best_tree_params = rs2.best_params_
print(f"Best tree parameters: {best_tree_params}")

Best tree parameters: {'gamma': 0.06711966552658265, 'max_depth': 5, 'min_child_weight': 3}


In [13]:
# Tune stochastic components
xgb4 = XGBRegressor(
    n_estimators=best_ntrees,
    learning_rate=best_lr,
    **best_tree_params, 
    early_stopping_rounds=50, 
    eval_metric="rmse", 
    random_state=808)

# Fit model
xgb4.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], verbose=0)

# Third param dist
param_dist3 = {
    "subsample": uniform(0.5, 0.5),
    "colsample_bytree": uniform(0.5, 0.5) 
}

# Set up RandomizedSearchCV
rs3 = RandomizedSearchCV(
    xgb4, param_dist3, n_iter=20, scoring='neg_root_mean_squared_error', 
    cv=3, verbose=False, random_state=808, n_jobs=10
)

# Run random search
rs3.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], verbose=0)

# Print best stochastic parameters
best_stochastic_params = rs3.best_params_
print(f"Best stochastic parameters: {best_stochastic_params}")

Best stochastic parameters: {'colsample_bytree': 0.6673574366507202, 'subsample': 0.8046025228990683}


In [3]:
test_df.columns

Index(['id', 'Lat_Dec', 'Lon_Dec', 'NO2uM', 'NO3uM', 'NH3uM', 'R_TEMP',
       'R_Depth', 'R_Sal', 'R_DYNHT', 'R_Nuts', 'R_Oxy_micromol.Kg', 'PO4uM',
       'SiO3uM', 'TA1', 'Salinity1', 'Temperature_degC'],
      dtype='object')

In [16]:
# Assign features
X_test= test_df
# y_test = test_df['DIC'] There is no DIC in the test df


# Initialize fifth XGB model
xgb5 = XGBRegressor(
    n_estimators=best_ntrees,
    learning_rate=best_lr,
    **best_tree_params, 
    **best_stochastic_params, 
    early_stopping_rounds=50, 
    eval_metric="rmse", 
    random_state=808)

# Fit model
xgb5.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], verbose=0)

# Predict on testing data
y_pred = xgb5.predict(X_val_scaled)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_val, y_pred)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

Mean Squared Error (MSE): 47.4414
Root Mean Squared Error (RMSE): 6.8878


In [17]:

# Get feature importance
feature_importance = pd.DataFrame({'Feature': X_train_scaled.columns, 'Importance': xgb5.feature_importances_})

# Sort by importance
feature_importance = feature_importance.sort_values(by="Importance", ascending=False)
feature_importance

,Feature,Importance
12,PO4uM,0.494485
13,SiO3uM,0.370204
11,R_Oxy_micromol.Kg,0.082770
7,R_Depth,0.015209
8,R_Sal,0.011374
14,TA1.x,0.011063
4,NO3uM,0.010024
15,Salinity1,0.003528
6,R_TEMP,0.000303
3,NO2uM,0.000191
